### install and import baseline dependencies

In [ ]:
!pip install transformers

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

### setup summarization model

In [ ]:
model_name =  "human-centered-summarization/financial-summarization-pegasus"
tokenizer_p =  PegasusTokenizer.from_pretrained (model_name)
model_p =  PegasusForConditionalGeneration.from_pretrained (model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at human-centered-summarization/financial-summarization-pegasus and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Summarize a single article

In [ ]:
url = "https://finance.yahoo.com/quote/HDB?.tsrc=fin-srch"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [ ]:
paragraphs

[<p class="P(20px) M(0)"><span>We are experiencing some temporary issues. The market data on this page is currently delayed. Please bear with us as we address this and restore your personalized lists.</span></p>,
 <p class="D(b)--mobp IbBox Fw(b) Fz(xs) Whs(n) Maw(175px)">Gap Up</p>,
 <p class="Fw(500) My(15px) Ta(c)">Subscribe to Yahoo Finance Plus to view Fair Value for HDB</p>,
 <p class="Fz(14px) Lh(19px) Fz(13px)--sm1024 Lh(17px)--sm1024 LineClamp(2,38px) LineClamp(2,34px)--sm1024 M(0)">In this piece, we will take a look at the 13 best bank stocks to invest in for long term. If you want to skip our coverage of the banking industry and how it started to trouble investors once again this year, you can take a look at 5 Best Bank Stocks To Invest In […]</p>,
 <p class="Fz(14px) Lh(19px) Fz(13px)--sm1024 Lh(17px)--sm1024 LineClamp(2,38px) LineClamp(2,34px)--sm1024 M(0)">There are thousands of listed stocks in the United States across sectors. However, portfolio diversification does not

In [ ]:
r.text

'<!doctype html><html data-color-theme="light" id="atomic" class="NoJs chrome desktop failsafe" lang="en-US"><head prefix="og: https://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark(\'PageStart\');</script><meta charset="utf-8"><title>HDFC Bank Limited (HDB) Stock Price, News, Quote &amp; History - Yahoo Finance</title><meta name="keywords" content="HDB, HDFC Bank Limited, HDB stock chart, HDFC Bank Limited stock chart, stock chart, stocks, quotes, finance"><meta http-equiv="x-dns-prefetch-control" content="on"><meta property="twitter:dnt" content="on"><meta property="fb:app_id" content="458584288257241"><meta name="theme-color" content="#400090"><meta name="viewport" content="width=device-width, initial-scale=1"><meta name="description" lang="en-US" content="Find the latest HDFC Bank Limited (HDB) stock quote, history, news and other vital information to help you with your stock trading and investing."><meta name="oath:guce:consent-host" c

In [ ]:
paragraphs = soup.find_all('p')

In [ ]:
paragraphs[0].text

'We are experiencing some temporary issues. The market data on this page is currently delayed. Please bear with us as we address this and restore your personalized lists.'

In [ ]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)

In [ ]:
len(words)

400

In [ ]:
ARTICLE

"We are experiencing some temporary issues. The market data on this page is currently delayed. Please bear with us as we address this and restore your personalized lists. Gap Up Subscribe to Yahoo Finance Plus to view Fair Value for HDB In this piece, we will take a look at the 13 best bank stocks to invest in for long term. If you want to skip our coverage of the banking industry and how it started to trouble investors once again this year, you can take a look at 5 Best Bank Stocks To Invest In […] There are thousands of listed stocks in the United States across sectors. However, portfolio diversification does not only imply exposure to multiple U.S. companies. Diversification is also not complete if we consider blue-chip and growth stocks of U.S.-domiciled companies. With a significant difference in GDP growth between emerging and developed economies, it’s important to look at investing in companies that provide global growth exposure. Investing outside the United States might sound 

In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# # Initialize tokenizer and model
# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# model = T5ForConditionalGeneration.from_pretrained('t5-small')


# Tokenize the article
input_ids = tokenizer_p.encode(ARTICLE, return_tensors='pt')

# Generate summary
output = model_p.generate(input_ids, max_length=95, num_beams=5, early_stopping=True)
summary = tokenizer_p.decode(output[0], skip_special_tokens=True)

# print(summary)

In [ ]:
summary

'HDFC Bank has raised $300 million through its debut dollar-denominated bond.'

### Building a news and sentiment pipeline

In [ ]:
monitored_tickers = ['HDB']

### Search for stock news using google and yahoo finance

In [ ]:
search_url = "https://finance.yahoo.com/quote/HDB/news/"
search_url


'https://finance.yahoo.com/quote/HDB/news/'

In [ ]:
def check_urls_existence(urls):
    existing_urls = []
    non_existing_urls = []

    for url in urls:
        try:
            response = requests.head(url)
            if response.status_code == 200:
                existing_urls.append(url)
            else:
                non_existing_urls.append(url)
        except Exception as e:
            print(f"Error checking URL: {url} - {e}")
            non_existing_urls.append(url)

    return existing_urls, non_existing_urls

search_url = "https://finance.yahoo.com/quote/HDB/news/"
raw_urls = [search_url]
existing_urls, non_existing_urls = check_urls_existence(raw_urls)

print("Existing URLs:")
print(existing_urls)
print("\nNon-existing URLs:")
print(non_existing_urls)


Existing URLs:
['https://finance.yahoo.com/quote/HDB/news/']

Non-existing URLs:
[]


In [ ]:
pip install selenium webdriver-manager


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup

def search_for_stock_news_urls(tickers):
    search_url = "https://finance.yahoo.com/quote/{}/news/".format(tickers)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = requests.get(search_url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            atags = soup.find_all('a', href=True)
            hrefs = [link['href'] for link in atags if 'href' in link.attrs]
            return hrefs
        else:
            print("Failed to retrieve data from Yahoo Finance.")
            return []
    except Exception as e:
        print("Error occurred while retrieving data:", e)
        return []

raw_urls = search_for_stock_news_urls('HDB')
raw_urls



['https://www.yahoo.com/',
 'https://mail.yahoo.com/',
 'https://news.yahoo.com/',
 'https://finance.yahoo.com/',
 'https://sports.yahoo.com/',
 'https://www.yahoo.com/entertainment/',
 'https://search.yahoo.com/search/',
 'https://mobile.yahoo.com/',
 'https://www.yahoo.com/everything/',
 'https://finance.yahoo.com',
 '#Nav-0-DesktopNav',
 '#market-summary',
 '#Aside',
 'https://login.yahoo.com/config/login?.src=finance&.intl=us&.lang=en-US&.done=https%3A%2F%2Ffinance.yahoo.com%2Fquote%2FHDB%2Fnews%2F&activity=uh-signin&pspid=95993639',
 'https://mail.yahoo.com/?.intl=us&.lang=en-US&.partner=none&.src=finance&activity=uh-mail&pspid=95993639',
 'https://login.yahoo.com/config/login?.src=finance&.intl=us&.lang=en-US&.done=https://finance.yahoo.com&activity=uh-mail&pspid=95993639',
 '/watchlists/',
 '/portfolios/',
 '/calendar/',
 '/news/',
 '/videos/',
 '/plus-dashboard?ncid=dcm_306158762_490172245_127172993',
 '/screener/',
 '/topic/personal-finance/',
 '/crypto/',
 '/sectors',
 'https

### Strip out Unwanted urls

In [ ]:
import re

In [ ]:
def filter_unwanted_urls(raw_urls):
    relevant_urls = []
    exclude_list = ['maps', 'microsoft', 'images', 'login']

    for url in raw_urls:
        # Check if the URL contains "hdb" or "HDB" and does not contain excluded words
        if ('hdb' in url.lower() or 'HDB' in url) and not any(exclude_word in url for exclude_word in exclude_list):
            relevant_urls.append(url)

    return relevant_urls

strip_out_unwanted_urls = filter_unwanted_urls(raw_urls)
strip_out_unwanted_urls


['/chart/HDB?studies=Visitor%20Trend&ncid=dcm_306158729_490172245_127172993',
 '/quote/HDB',
 '/quote/HDB/company-insights',
 '/quote/HDB/chart',
 '/quote/HDB/community',
 '/quote/HDB/key-statistics',
 '/quote/HDB/history',
 '/quote/HDB/profile',
 '/quote/HDB/financials',
 '/quote/HDB/analysis',
 '/quote/HDB/options',
 '/quote/HDB/holders',
 '/quote/HDB/sustainability',
 '/chart/HDB?showOptin=1',
 '/chart/HDB?technical=all&ncid=dcm_306158732_490172245_127172993',
 '/chart/HDB?technical=short&ncid=dcm_306158732_490172245_127172993',
 '/chart/HDB?technical=intermediate&ncid=dcm_306158732_490172245_127172993',
 '/chart/HDB?technical=long&ncid=dcm_306158732_490172245_127172993',
 '/research/stock-forecast/HDB?symbols=HDB&ncid=dcm_306158744_490172245_127172993',
 '/quote/HDB/',
 '/quote/HDB/press-releases',
 '/quote/HDB/reports',
 '/quote/HDB/sec-filing',
 '/news/hdfc-bank-limited-hdb-benefitted-110217001.html']

In [ ]:
# def filter_unwanted_urls(raw_urls):
#     relevant_urls = []
#     exclude_list = ['maps', 'microsoft', 'images', 'login']

#     for url in raw_urls:
#         # Check if the URL starts with "https://" and does not contain excluded words
#         if url.startswith("https://") and not any(exclude_word in url for exclude_word in exclude_list):
#             relevant_urls.append(url)

#     return relevant_urls

# strip_out_unwanted_urls = filter_unwanted_urls(raw_urls)
# strip_out_unwanted_urls

### Search and Scrape cleaned urls

In [ ]:
import time  # Import the time module for adding delays between retries

def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs:
        # Check if the URL has the schema included, if not, prepend 'https://' as default
        if not url.startswith('http://') and not url.startswith('https://'):
            url = 'https://finance.yahoo.com' + url

        try:
            r = requests.get(url)
            if r.status_code == 200:  # Check if the request was successful
                soup = BeautifulSoup(r.text, 'html.parser')
                paragraphs = soup.find_all('p')
                text = [paragraph.text for paragraph in paragraphs]
                words = ' '.join(text).split(' ')[:400]
                ARTICLE = ' '.join(words)
                ARTICLES.append(ARTICLE)
            else:
                print(f"Skipping URL: {url} - Status code: {r.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error accessing URL: {url} - {e}")
        except Exception as e:
            print(f"Unknown error occurred for URL: {url} - {e}")

        # Add a delay between requests to avoid overwhelming the server
        time.sleep(1)  # Add a 1-second delay between requests

    return ARTICLES

# Example usage
articles = scrape_and_process(raw_urls)
articles

Skipping URL: https://finance.yahoo.com/videos/ - Status code: 400
Skipping URL: https://finance.yahoo.com/plus-dashboard?ncid=dcm_306158762_490172245_127172993 - Status code: 404
Skipping URL: https://finance.yahoo.com/chart/HDB?studies=Visitor%20Trend&ncid=dcm_306158729_490172245_127172993 - Status code: 404
Skipping URL: https://finance.yahoo.com/quote/HDB/company-insights - Status code: 404
Skipping URL: https://finance.yahoo.com/quote/HDB/chart - Status code: 404
Skipping URL: https://finance.yahoo.com/quote/HDB/community - Status code: 404
Skipping URL: https://finance.yahoo.com/quote/HDB/key-statistics - Status code: 404
Skipping URL: https://finance.yahoo.com/quote/HDB/history - Status code: 404
Skipping URL: https://finance.yahoo.com/quote/HDB/profile - Status code: 404
Skipping URL: https://finance.yahoo.com/quote/HDB/financials - Status code: 404
Skipping URL: https://finance.yahoo.com/quote/HDB/analysis - Status code: 404
Skipping URL: https://finance.yahoo.com/quote/HDB/op

["The survey shows how powerful politicians — particularly Donald Trump — can sway the public when they take sides on a key issue. Republicans trust Trump on immigration\xa0» Follow all the developments right here with Yahoo Sports. Senate Majority Leader Chuck Schumer of New York, the highest ranking Jewish member of the U.S. government, delivered an extraordinary speech on the floor of the U.S. Senate on Thursday that criticized Israeli Prime Minister Benjamin Netanyahu’s handling of the war in Gaza and called for new elections in Israel. A bill that could lead to a national TikTok ban cleared the House on March 13 and heads to the Senate.  The probe of the Eagles stems from comments made by Penn State head coach James Franklin. Cousins' investigation stems from comments he made in his introductory news conference. Rodgers' statement comes a day after a CNN story detailing comments Rodgers made about the shooting in 2013. You don't have to splurge to give a great gift —\xa0we found a

### Summarize all articles

In [ ]:
def summarize(articles):
    summaries=[]
    for article in articles:
        input_ids = tokenizer.encode(article,return_tensors='pt')
        output = model.generate(input_ids, max_length=205, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries


In [ ]:
summaries = summarize(articles)
summaries

["the Eras Tour (Taylor's Version)'. Waymo will begin offering a robotaxi service to the public in Los Angeles this week and in Austin by the end of the year, the company's co-CEO said.",
 'Yahoo Mail.',
 'a woman who allegedly ran a retail theft ring that targeted stores nationwide. a florida woman was able to drive away with more than 7,000 gallons of gas for free over six months. a team of Fox News employees, including Oleksandra "Sasha" Kuvshynova, pushed into a war zone despite warnings to stay out.',
 'voted in favor of the clinical benefit/risk profile of imetelstat for the treatment of transfusion-dependent anemia. KE Holdings, or Beike, is a large residential real estate sales and rental brokerage company in China. KE Holdings, or Beike, is a large residential real estate sales and rental brokerage company in China.',
 "took it as compliment, even though it didn't really sound like one. The Bears make two of the top three picks. J.J. McCarthy goes top five. One of the best pro

### Adding Sentiment Analysis

In [ ]:
from transformers import pipeline

In [ ]:
sentiment = pipeline ('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
scores = sentiment(summaries)
scores

[{'label': 'POSITIVE', 'score': 0.9902852177619934},
 {'label': 'NEGATIVE', 'score': 0.9000168442726135},
 {'label': 'NEGATIVE', 'score': 0.9932332634925842},
 {'label': 'POSITIVE', 'score': 0.669917106628418},
 {'label': 'POSITIVE', 'score': 0.9935383200645447},
 {'label': 'NEGATIVE', 'score': 0.9771840572357178},
 {'label': 'POSITIVE', 'score': 0.7481208443641663},
 {'label': 'POSITIVE', 'score': 0.9993513226509094},
 {'label': 'POSITIVE', 'score': 0.7481208443641663},
 {'label': 'POSITIVE', 'score': 0.669917106628418},
 {'label': 'POSITIVE', 'score': 0.669917106628418},
 {'label': 'POSITIVE', 'score': 0.669917106628418},
 {'label': 'POSITIVE', 'score': 0.669917106628418},
 {'label': 'NEGATIVE', 'score': 0.9988467693328857},
 {'label': 'NEGATIVE', 'score': 0.9000168442726135},
 {'label': 'NEGATIVE', 'score': 0.9988467693328857},
 {'label': 'NEGATIVE', 'score': 0.8575285077095032},
 {'label': 'NEGATIVE', 'score': 0.9658610820770264},
 {'label': 'NEGATIVE', 'score': 0.5148525238037109}